# Notes

In [ ]:
## Point selection after nearest location isn't random, defaults to lowest value index

# Imports

In [ ]:
##IMPORTS
import numpy as np
from numpy import *
import copy

# Take in or create input

In [ ]:
## FOR THE BIG 600 CELL ONE
import csv

with open("UseThisOne.csv") as file_name:
    inputMatrix = np.loadtxt(file_name, dtype = 'int', delimiter=",")

    
## FOR SMALLER TESTING SOLIDS
inputMatrix = array([[1,4,5,8],
                [1,4,6,8],
                [1,3,6,8],
                [1,3,5,8],
                [2,3,5,8],
                [2,4,5,8],
                [2,4,6,8],
                [2,3,6,8],
                [1,4,5,7],
                [1,4,6,7],
                [1,3,6,7],
                [1,3,5,7],
                [2,3,5,7],
                [2,4,5,7],
                [2,4,6,7],
                [2,3,6,7]
               
               ])



    
print(inputMatrix)


# Create master array for which faces have and haven't been covered

In [ ]:
## Create Master Array of Uncovered Faces
## Will be removed from throughout process
masterUncoveredFaces = copy.deepcopy(inputMatrix)
#print(masterUncoveredFaces)

##Create Master Array for covered faces, starts empty, fills to be inputMatrix
rows, cols = (1, len(inputMatrix[0]))
masterCoveredFaces = [[-1 for i in range(cols)] for j in range(rows)]
#print(masterCoveredFaces)

# Find total number of vertices

In [ ]:
## Find number of vertices
inputMatrixWidth= len(inputMatrix[0])
maxVert = 0
for x in inputMatrix:
    if x[inputMatrixWidth-1] > maxVert:
        maxVert = x[inputMatrixWidth-1]

print(maxVert)

# Creating the Dictionary of each face that touches each vertex

In [ ]:
## Create list of faces touched by each vertex, as well as copy for strictly uncovered

## How to know how many faces each vertex will touch? **************************
    ## For platonic solid, equal for all, so find max faces for one, for all
    ## Just make it a dictionary, probably better
   

vertexDictOfTouchedFaces = {
    
}

for nextVertex in range(maxVert):
    vertexDictOfTouchedFaces[nextVertex+1] = []
    
## Fill in the dictionary
for currentFace in inputMatrix:
    for currentVertex in currentFace:
            vertexDictOfTouchedFaces[currentVertex].append(currentFace)
            
#print(vertexDictOfTouchedFaces)


##Create the dictionary for each uncovered face, will be removed from throughout the process
vertexDictOfTouchingUncoveredFaces = copy.deepcopy(vertexDictOfTouchedFaces)

#print(vertexDictOfTouchedFaces[8])
    

# Create Dictionary Of Which Vertices a Single Vertex shares edges with

In [ ]:
##Create something that stores every vertex, and each other vertex it shares an edge with
    ## shares edge if it shares 2 faces
    ##Look in vertexDictOfTouchedFaces, keep count of each time each vertex appears. If > 1 and not the original, edge
##What to store in, dict doesn't seem to be working well, hard with arrays as values
    ## Dict for now
    
vertexDictOfSharedEdges = {}

for nextVertex in range(maxVert):
    vertexDictOfSharedEdges[nextVertex+1] = []

vertexDictOfVerticesOnEdge = {    
}

for nextVertex in range(maxVert):
    vertexDictOfVerticesOnEdge[nextVertex+1] = []

rows, cols = (maxVert, maxVert)
vertexSharedFacesCount = [[0 for i in range(cols)] for j in range(rows)]
#print(len(vertexSharedFacesCount)*len(vertexSharedFacesCount[0]))



for currentVertex in vertexDictOfTouchedFaces:
    
    #print(currentVertex, vertexDictOfTouchedFaces[currentVertex])
    for currentFace in vertexDictOfTouchedFaces[currentVertex]:
        for vertexOnFace in currentFace:
            vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] += 1
            if vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] > 1 and vertexOnFace not in vertexDictOfSharedEdges[currentVertex]:
                    if vertexOnFace != currentVertex:
                        vertexDictOfSharedEdges[currentVertex].append(vertexOnFace)
    
    ##PUT THIS INSIDE LOOP ABOVE, SO TO ACCESS THE VALUE OF vertexOnFace
    ## Also, use number of faces each vertex is on to eliminate vertices pairing wiht self
    ##for x in vertexSharedFacesCount[currentVertex-1]:
        ##if x > 1:
            ##print(x, currentVertex)
            


#print(vertexSharedFacesCount)
#print(vertexDictOfSharedEdges)

tempVertexDictOfSharedEdges = copy.deepcopy(vertexDictOfSharedEdges)
#print(tempVertexDictOfSharedEdges)

#Once this removes an element from the dictionary, it doesn't check the next value. use temp dictionary
for currentVertex in vertexDictOfSharedEdges:
    for connectedVertex in vertexDictOfSharedEdges[currentVertex]:
        #print(currentVertex, connectedVertex)
        if connectedVertex == currentVertex:
            tempVertexDictOfSharedEdges[currentVertex].remove(connectedVertex)
            #print("removed", connectedVertex)
            
#print(vertexDictOfSharedEdges)

vertexDictOfSharedEdges = copy.deepcopy(tempVertexDictOfSharedEdges)
    
#print(vertexDictOfSharedEdges)  



# Fill out matrix of vertex to vertex distances

In [ ]:
rows, cols = (maxVert, maxVert)
vertexToVertexDistance = [[999 for i in range(cols)] for j in range(rows)]
#print(vertexToVertexDistance)

for i in range(0,maxVert):
    vertexToVertexDistance[i][i] = 0

#print( vertexToVertexDistance)

edgesAway = 1

while np.amax(vertexToVertexDistance) == 999:
    for vertex in vertexDictOfSharedEdges:
        for secondVertex in vertexDictOfSharedEdges:
            if vertexToVertexDistance[vertex-1][secondVertex-1] == edgesAway-1:
                #print(secondVertex, " is ", edgesAway, " away")
                for vertXAway in vertexDictOfSharedEdges[secondVertex]:
                    if vertexToVertexDistance[vertex-1][vertXAway-1] == 999:
                        vertexToVertexDistance[vertex-1][vertXAway-1] = edgesAway
    edgesAway += 1
#print(vertexToVertexDistance)
  
#print(vertexToVertexDistance)
#print(np.amax(vertexToVertexDistance))

# Calculate the distance from each maxUntouchedVertex to a covered face

In [ ]:
## Create function for running calucations on each vertex in an input list
## Simply returns which of the input vertices is closest

## Premake the matrix of distances

## When called, search through covered faces
## Pull out every vertex in the covered faces sets

## For each maxUntouchedVertex, find distance each vertex in covered faces sets, take min
## Store min as distance from covered faces

## Find maxUntouchedVertex with smallest distance from covered faces

## Return that maxUntouchedVertex

def findNextVertex(listOfMaxedFaceVerts):
    
    ## Fill out all vert touching a covered face
    verticesTouchingCoveredFaces = []
    for coveredFace in masterCoveredFaces:
        for vertex in coveredFace:
            if vertex not in verticesTouchingCoveredFaces:
                #print( vertex, "isn't in verticesTouchingCoveredFaces")
                verticesTouchingCoveredFaces.append(vertex)
                #print("verticesTouchingCoveredFaces is now ", verticesTouchingCoveredFaces)
    dictOfMaxedVertDistanceToFace = {}            
    for vertex in maxUntouchedVerts:
        dictOfMaxedVertDistanceToFace[vertex] = []
        closestDistance = 999999
        for vertOnFace in verticesTouchingCoveredFaces:
            if vertexToVertexDistance[vertex-1][vertOnFace -1] < closestDistance:
                closestDistance = vertexToVertexDistance[vertex-1][vertOnFace -1]
        #print(vertex, closestDistance)
        dictOfMaxedVertDistanceToFace[vertex] = closestDistance
            
        
    #print(dictOfMaxedVertDistanceToFace)

    ## Find maxUntouchedVertex with smallest distance from covered faces
    closestVert = 9999
    closestVertDist = 9999
    
    for vertex in dictOfMaxedVertDistanceToFace:
        if dictOfMaxedVertDistanceToFace[vertex] < closestVertDist:
            closestVert = vertex
            closestVertDist = dictOfMaxedVertDistanceToFace[vertex]
    return(closestVert)

# Randomly select starting vertex

In [ ]:
## Select Random Starting vertex
startingVertex = random.randint(1,maxVert+1)
lastVisitedVertex = copy.deepcopy(startingVertex)
solutionVertices = list()


In [ ]:
#startingVertex = 1
print("We are starting at " , startingVertex)
continueAddingVertices = True

while continueAddingVertices == True:
    
    
    ## Add vertex to a solution attempt
    solutionVertices.append(lastVisitedVertex)
    #print(solutionVertices)
    
    
    # Processes Faces Touched by current vertex
    # Counts faces as covered for master array
    # Removes face from each vertex's untouched faces dictionary
    
    ## Find touched faces
    currentTouchedFaces = vertexDictOfTouchedFaces[lastVisitedVertex]
    #print(currentTouchedFaces)


    ## Count all as "covered"
        ## Add to a set of "covered faces"
    ##Needed to set size of array, make sure to remove later!!!!!!!!!!!!!!!!!!!!!!!!
    facesToBeAdded = np.ndarray(shape = (1, len(inputMatrix[0])))
    facesToBeAdded.fill(-1)
    facesToBeAdded = facesToBeAdded.astype(int)
    for x in currentTouchedFaces:
        facesToBeAdded = np.vstack([facesToBeAdded, x])

    #print(facesToBeAdded)

    masterCoveredFaces = np.vstack([masterCoveredFaces, facesToBeAdded])

    ##Removes the size setting rows
    removed = (masterCoveredFaces == -1).sum(1)
    fixed = masterCoveredFaces[removed == 0, :]


    masterCoveredFaces = copy.deepcopy(fixed)

    #print(masterCoveredFaces)



    ## Remove from the current vertex's list of uncovered faces
    for x in vertexDictOfTouchingUncoveredFaces:
        ##make temp
        tempXTouchingUncovered = np.ndarray(shape = (1, len(inputMatrix[0])))
        tempXTouchingUncovered.fill(-1)
        tempXTouchingUncovered = tempXTouchingUncovered.astype(int)
        for y in vertexDictOfTouchingUncoveredFaces[x]:
            ## make inZ
            inZ = False
            for z in vertexDictOfTouchedFaces[lastVisitedVertex]:
                if np.array_equal(y,z):
                    #Set inZ to true
                    inZ = True
                    #print("These are equal")
            ## if not inZ, add to temp
            if not inZ:
                tempXTouchingUncovered = np.vstack([tempXTouchingUncovered, y])
        ## fix temp
            #print(tempXTouchingUncovered)
            removed = (tempXTouchingUncovered == -1).sum(1)
            fixed = tempXTouchingUncovered[removed == 0, :]
            #print("Temp list ", x, fixed)
            tempXTouchingUncovered = copy.deepcopy(fixed)
        
        
        removed = (tempXTouchingUncovered == -1).sum(1)
        fixed = tempXTouchingUncovered[removed == 0, :]
        #print(x, fixed)
        tempXTouchingUncovered = copy.deepcopy(fixed)

        ## make temp x's uncovered touching
        #print("Our temp stuff is: ", x,tempXTouchingUncovered)
        vertexDictOfTouchingUncoveredFaces[x] = copy.deepcopy(tempXTouchingUncovered)
        #print("this",vertexDictOfTouchingUncoveredFaces[x])



    #print(vertexDictOfTouchingUncoveredFaces[lastVisitedVertex])
    
    
    
    
    
    
    # Removes faces touched by current vertex from master uncovered array
    ## Remove touched faces from "uncovered" list of faces
    ## Removing is hard when not sure which row location removing from.
    ## Maybe try just rewriting without the covered ones?

    #tempUncoveredFaces = array([[-1,-1,-1,-1]])
    tempUncoveredFaces = np.ndarray(shape = (1, len(inputMatrix[0])))
    tempUncoveredFaces.fill(-1)
    tempUncoveredFaces = tempUncoveredFaces.astype(int)
    #print(vertexDictOfTouchingUncoveredFaces)

    for x in masterUncoveredFaces:
        inY = False
        for y in vertexDictOfTouchedFaces[lastVisitedVertex]:
            #print(vertexDictOfTouchedFaces[lastVisitedVertex])
            if np.array_equal(x,y):
                inY = True
        if not inY:
            tempUncoveredFaces = np.vstack([tempUncoveredFaces,x])
    #print(tempUncoveredFaces)

    removed = (tempUncoveredFaces == -1).sum(1)
    fixed = tempUncoveredFaces[removed == 0, :]

    masterUncoveredFaces = copy.deepcopy(fixed)

    print("Faces Not Yet Covered") 
    print(masterUncoveredFaces)
    print("")
    print("Faces covered") 
    print(masterCoveredFaces)
    print("")
    
      ## If yes, output list of vertices as solution
    ## Covered face must be 1 version of all, and uncovered must be empty
    if len(masterUncoveredFaces) == 0:
        print("Solution Set")
        print(solutionVertices)
        print(len(solutionVertices), " vertices in solution set")
        print( " ")
        print("Uncoverd faces")
        print(masterUncoveredFaces)
        print( " ")
        print("Covered Faces")
        print(masterCoveredFaces)
        continueAddingVertices = False
        break
    
    
    #print(lastVisitedVertex)
    ## If no, select next Vertex
        
        
    # Find all vertices with highest uncovered faces
    
    ## Finding Next Vertex
    ## Find all vertices that touch maximum number of uncovered faces
    
    #print(vertexDictOfTouchingUncoveredFaces)
    
    maxUntouched = 0
    maxUntouchedVerts = []
    for x in vertexDictOfTouchingUncoveredFaces:
        if len(vertexDictOfTouchingUncoveredFaces[x]) > maxUntouched:
            maxUntouched = len(vertexDictOfTouchingUncoveredFaces[x])
        #print(x, len(vertexDictOfTouchingUncoveredFaces[x]))
        #print("max ", maxUntouched )

    for x in vertexDictOfTouchingUncoveredFaces:
        if len(vertexDictOfTouchingUncoveredFaces[x]) == maxUntouched:
                maxUntouchedVerts.append(x)
    print("Vertices with maximum untouched faces, [", maxUntouched, "] are: ",maxUntouchedVerts)
    #print(maxUntouchedVerts[0])


        ## Calculate which of vertices in previous step are closest to a covered face 
    if len(maxUntouchedVerts) == 1:
        lastVisitedVertex = maxUntouchedVerts[0]
    elif len(maxUntouchedVerts) == 0:
        print("SOMETHING IS VERY WRONG HERE, NO MAXIMIZED VERTS")
    else:
        lastVisitedVertex = findNextVertex(maxUntouchedVerts)
        print("We are now going to ",lastVisitedVertex)


    #print("The next vertex is" , lastVisitedVertex)

    
    # Check if all faces covered, if yes, print out solution, covered and uncovered
    ## Check if all faces covered
    #print(inputMatrix)
    #print(masterCoveredFaces)
    #print(np.array_equal(inputMatrix, masterCoveredFaces))

  
